In [3]:
# set file path
filepath = '../data/'
trainfile = 'train.csv'

In [4]:
# read train.csv
import pandas as pd
df = pd.read_csv(filepath+trainfile)

In [66]:
import re

df['hashtag'] = df.text.apply(lambda doc: ' '.join(re.findall(r'(?<=[\s^])#\w+(?=[\s$])', doc)))
df['url'] = df.text.apply(
    lambda doc: ' '.join(re.findall(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', doc))
)

df['hashtag_and_url'] = df.hashtag.str.cat(df.url.apply(lambda url: ' %s' % url))

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=1)
X_train = cv.fit_transform(df.hashtag_and_url.values)
X_train.shape

(7613, 5850)

In [67]:
pd.DataFrame(X_train.todense(), columns=['url_%s' % column for column in cv.get_feature_names()])

,url_007npen6lg,url_00cy9vxeff,url_0215,url_02elqlopfk,url_034,url_03l7nwqdje,url_06,url_073izwx0lb,url_08lngclzsj,url_09,...,url_zy3hpdjnwg,url_zyrzx6dfki,url_zyssobxntc,url_zz0cewew64,url_zz5udilrea,url_zzcbbqyj1q,url_zzpojgngaj,url_zzsee5hipm,url_zztbvjypn1,url_zzweeezjug
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7610,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7611,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df[['keyword', 'location']].values, df.target, stratify=df.target.values, random_state=0, test_size=0.1
)

In [58]:
import category_encoders as ce

encoder = ce.CatBoostEncoder()
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)
X_test = encoder.transform(X_test)
X_train

,0,1
0,0.202477,0.429718
1,0.453815,0.426680
2,0.456381,0.204245
3,0.153089,0.429718
4,0.562249,0.429718
...,...,...
6846,0.345270,0.429718
6847,0.368701,0.489960
6848,0.180511,0.848807
6849,0.095270,0.429718


In [59]:
from catboost import CatBoost
from catboost import Pool

In [69]:
# CatBoost が扱うデータセットの形式に直す
train_pool = Pool(X_train, label=y_train)
test_pool = Pool(X_test, label=y_test)

# 学習用のパラメータ
params = {
    # タスク設定と損失関数
    'loss_function': 'Logloss',
    # 学習ラウンド数
    'num_boost_round': 100,
}

# モデルを学習する
model = CatBoost(params)
model.fit(train_pool)

# 検証用データを分類する
# NOTE: 確率がほしいときは prediction_type='Probability' を使う
#y_pred = model.predict(test_pool, prediction_type='Class')

Learning rate set to 0.22472
0:	learn: 0.6285945	total: 37.2ms	remaining: 3.68s
1:	learn: 0.5863191	total: 42.2ms	remaining: 2.06s
2:	learn: 0.5586983	total: 46.8ms	remaining: 1.51s
3:	learn: 0.5392643	total: 60ms	remaining: 1.44s
4:	learn: 0.5261553	total: 164ms	remaining: 3.11s
5:	learn: 0.5150717	total: 169ms	remaining: 2.65s
6:	learn: 0.5077988	total: 173ms	remaining: 2.3s
7:	learn: 0.5014417	total: 178ms	remaining: 2.05s
8:	learn: 0.4971865	total: 185ms	remaining: 1.87s
9:	learn: 0.4942250	total: 191ms	remaining: 1.71s
10:	learn: 0.4914493	total: 203ms	remaining: 1.65s
11:	learn: 0.4894134	total: 218ms	remaining: 1.6s
12:	learn: 0.4873646	total: 230ms	remaining: 1.54s
13:	learn: 0.4858947	total: 238ms	remaining: 1.46s
14:	learn: 0.4847533	total: 244ms	remaining: 1.38s
15:	learn: 0.4838653	total: 250ms	remaining: 1.31s
16:	learn: 0.4833210	total: 254ms	remaining: 1.24s
17:	learn: 0.4827301	total: 260ms	remaining: 1.18s
18:	learn: 0.4821844	total: 265ms	remaining: 1.13s
19:	learn: 0

In [70]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score

y_pred = model.predict(test_pool, prediction_type='Class')
y_prob = model.predict(test_pool, prediction_type='Probability')

print('f1:{}'.format(f1_score(y_test, y_pred)))
print(confusion_matrix(y_test, y_pred))

f1:0.7258566978193146
[[353  82]
 [ 94 233]]


In [71]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.81      0.80       435
           1       0.74      0.71      0.73       327

    accuracy                           0.77       762
   macro avg       0.76      0.76      0.76       762
weighted avg       0.77      0.77      0.77       762



In [ ]:
from joblib import dump

dump(bc, '../model/bc.pkl')
dump(cv, '../model/cv.pkl') 
dump(naive_bayes, '../model/nb.pkl') 